In [2]:
#coding=utf-8
from keras.models import Model
from keras.layers import Input,Dense,Dropout,BatchNormalization,Conv2D,MaxPooling2D,AveragePooling2D,concatenate,Activation,ZeroPadding2D
from keras.layers import add,Flatten
import numpy as np
seed = 7
np.random.seed(seed)

In [3]:
num_BN = 1

def Conv2d_BN(x, nb_filter,kernel_size, strides=(1,1), padding='same',name=None):
    if name is None:
        name = str(num_BN)
        num_BN += 1 
    conv_name = name + "_Conv"  
    bn_name = name + "_BN"
    x = Conv2D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu',name=conv_name)(x)
    x = BatchNormalization(axis=3,name=bn_name)(x)
    return x

def Conv_Block(inpt,nb_filter,kernel_size,strides=(1,1), with_conv_shortcut=False, block_no=None):
    block_name = "block"+str(block_no)
    x = Conv2d_BN(inpt,nb_filter=nb_filter,kernel_size=kernel_size,strides=strides,padding='same', name=block_name+"_1")
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size,padding='same',  name=block_name+"_2")
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt,nb_filter=nb_filter,strides=strides,kernel_size=kernel_size, name="short_block"+str(block_no))
        x = add([x,shortcut])
        return x
    else:
        x = add([x,inpt])
        return x

In [4]:
inpt = Input(shape=(224,224,3), name="Input")
x = ZeroPadding2D((3,3), name="ZeroPadding")(inpt)
x = Conv2d_BN(x,nb_filter=64,kernel_size=(7,7),strides=(2,2),padding='valid', name="NB1") 
x = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same', name="pool1")(x)
#(56,56,64)
x = Conv_Block(x,nb_filter=64,kernel_size=(3,3), block_no=1)
x = Conv_Block(x,nb_filter=64,kernel_size=(3,3), block_no=2)
x = Conv_Block(x,nb_filter=64,kernel_size=(3,3), block_no=3)
#(28,28,128)
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True, block_no=4)
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3), block_no=5)
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3), block_no=6)
x = Conv_Block(x,nb_filter=128,kernel_size=(3,3), block_no=7)
#(14,14,256)
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True, block_no=8)
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3), block_no=9)
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3), block_no=10)
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3), block_no=11)
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3), block_no=12)
x = Conv_Block(x,nb_filter=256,kernel_size=(3,3), block_no=13)
#(7,7,512)
x = Conv_Block(x,nb_filter=512,kernel_size=(3,3),strides=(2,2),with_conv_shortcut=True, block_no=14)
x = Conv_Block(x,nb_filter=512,kernel_size=(3,3), block_no=15)
x = Conv_Block(x,nb_filter=512,kernel_size=(3,3), block_no=16)
x = AveragePooling2D(pool_size=(7,7), name="AveragePooling2D1")(x)
x = Flatten(name="Flat1")(x)
x = Dense(1000,activation='softmax', name="Dense1")(x)
 
model = Model(inputs=inpt,outputs=x)
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.summary()

# 把模型保存为图片
from keras.utils import plot_model
plot_model(model,to_file='model_png/206Resnet34.png', show_layer_names=True, show_shapes=True) 

W0808 17:07:47.029598 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 17:07:47.081192 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 17:07:47.159668 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 17:07:47.161434 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
ZeroPadding (ZeroPadding2D)     (None, 230, 230, 3)  0           Input[0][0]                      
__________________________________________________________________________________________________
NB1_Conv (Conv2D)               (None, 112, 112, 64) 9472        ZeroPadding[0][0]                
__________________________________________________________________________________________________
NB1_BN (BatchNormalization)     (None, 112, 112, 64) 256         NB1_Conv[0][0]                   
__________________________________________________________________________________________________
pool1 (Max